In [34]:
from langchain.vectorstores import Chroma 
from langchain_openai import OpenAIEmbeddings

In [35]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [29]:
import os
db_name = "test.db"
persist_directory = os.path.join("../vector_embedding", db_name)

In [36]:
vector_store = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(),
    persist_directory= persist_directory
)

In [37]:
vector_store.add_documents(documents)

['1a334a0b-6669-4f14-a970-5a53434dbcdb',
 '8e2c9a27-80b3-45ef-bee0-637defe360e6',
 '884f0d44-8fb6-45e6-9b6c-256dbb5b8620',
 '3cc66429-a94c-41bc-a9b0-e2f5f4a7bfbb',
 '3c6af7ac-9070-466a-a35d-a6c151080667']

In [39]:
vector_store.persist()

/tmp/ipykernel_28475/485603143.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()


In [38]:
from langchain_core.documents import Document
from langchain_core.prompts import  ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [18]:
retriever = vector_store.as_retriever(
    search_type = "similarity",
     search_kwargs={
         'k': 2,
         }
)

In [19]:
retriever.batch(["cats", "food"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]]

In [22]:
message ="""

Write more on the type of animals based on this context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

In [23]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [24]:
rag_chain = {"context": retriever,"question": RunnablePassthrough()} | prompt | model

In [26]:
response = rag_chain.invoke("tell me about cats")

print(response.content)

Based on the provided context, cats can be characterized as independent animals that typically prefer to have their own space. This independence sets them apart from other pets, such as dogs, which are known for their companionship and loyalty. 

Cats are often solitary hunters by nature, which contributes to their self-sufficient demeanor. They can entertain themselves for extended periods and may not require constant interaction or attention from their owners. This trait makes them suitable for individuals or families with busy lifestyles who may not be able to dedicate as much time to a pet.

Additionally, cats are known for their grooming habits and often spend a significant amount of time cleaning themselves. They are also playful animals, enjoying toys and engaging in activities that mimic hunting behavior, such as pouncing and chasing.

In terms of personality, cats can vary greatly. Some may be more social and enjoy interacting with humans, while others may be more reserved and